[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# Covid-19 Dashboard
This is DIY Covid Dashboard provides a quick visualisation of (new) daily COVID-19 cases in England, split by region.
All data is publicly available at [PHE dashboard](https://coronavirus.data.gov.uk/).

In [1]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

from uk_covid19.exceptions import FailedRequestError
from time import sleep

In [2]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

### 1. Load initial data from disk
Access saved json data from PHE.

In [3]:
# Load JSON files and store the raw data:
with open("All_regions.json", "rt") as INFILE:
    All_regions=json.load(INFILE)

### 2. Wrangle the data
Transforming json file into nice dataframe.

In [4]:
# Define function to create dataframe from pre-downloaded data:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(rawdata):
    dates2 = [x['date'] for x in rawdata['East Midlands']['data']]
    dates2.sort() # !!! It's ESSENTIAL that you sort it, otherwise it won't be able to create a time range later on!!!
    index2=pd.date_range(parse_date(dates2[0]), parse_date(dates2[-1]), freq='D')
    # Empty dataframe:
    df=pd.DataFrame(index=index2, columns=[x for x in rawdata])  
    #Populate dataframe from json data:
    for region in rawdata: # each entry is a dictionary with date, cases, hospital and deaths
        for entry in range(0,len(rawdata[region]['data'])):
            date=parse_date(rawdata[region]['data'][entry]['date'])
            #print(date)

            if pd.isna(df.loc[date, region]): 
                # replace None with 0 in our data 
                value= rawdata[region]['data'][entry]['New cases'] if rawdata[region]['data'][entry]['New cases']!=None else 0.0
                # this is the way you access a specific location in the dataframe - use .loc
                # and put index,column in a single set of [ ]
                df.loc[date, region]=value

    # fill in any remaining "holes" due to missing dates
    df.fillna(0.0, inplace=True)
    return df

In [5]:
# Create dataframe from pre-downloaded data:
df_to_plot = wrangle_data(All_regions)
#df_to_plot

,East Midlands,East of England,London,North East,North West,South East,South West,West Midlands,Yorkshire and the Humber
2020-04-17,169,360,704,263,594,552,309,373,315
2020-04-18,138,344,689,158,550,414,226,327,370
2020-04-19,236,273,453,280,634,678,221,382,257
2020-04-20,97,256,297,216,457,315,192,360,150
2020-04-21,120,235,418,164,460,778,184,330,251
...,...,...,...,...,...,...,...,...,...
2021-10-22,3394,4796,3928,2037,5089,7347,6050,4295,3997
2021-10-23,3642,5353,4435,2353,4559,7365,6131,3698,3533
2021-10-24,2558,3618,3279,1610,3778,6250,4832,2838,3025
2021-10-25,2897,3891,3118,1430,3879,6258,4863,3465,2904


### 4. Download current data

Saved data contains updated information since the beginning of the pandemic (2020-04-17) up to:

In [6]:
print('COVID data up to: ' + str(df_to_plot.index[-1]).split(' ')[0])

COVID data up to: 2021-10-26


If you wish to get the most up-to-date version of the data, please click the following button to download the data directly from PHE.
This process may take a few minutes. Please wait for the confirmation icon and green colour.

In [7]:
# Place your API access code in this function. Do not call this function directly; it will be called by 
# the button callback. 
def access_api():
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """    
    output={}
    failures=[]
    region=['East Midlands', 'East of England', 'London', 'North East', 'North West', 'South East', 'South West', 'West Midlands', 'Yorkshire and the Humber']

    for x in region:
        #print('areaName='+x)
        filters = [
        'areaType=region',
        'areaName='+x]
     # values here are the names of the PHE metrics
        structure = {
            "date":"date",
            "Total cases":"cumCasesByPublishDate",
            "New cases":"newCasesByPublishDate",
            "New tests":"newAntibodyTestsByPublishDate"}
        api = Cov19API(filters=filters, structure=structure)
        try:
            output[x]=api.get_json()
        except FailedRequestError as e:
            failures.append(x)
        print(x)

        sleep(5)

    print("Update Finished!")
    return output, failures # return data read from the API --> this will return a tuple [0]=output and [1]=failures (remember when calling info)

In [8]:
# Printout from this function will be lost in Voila unless captured in an
# output widget - therefore, we give feedback to the user by changing the 
# appearance of the button
def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    # Get fresh data from the API. If you have time, include some error handling
    # around this call.
    updated_data=access_api()
    # wrangle the data and overwrite the dataframe for plotting
    global df_to_plot
    df_to_plot=wrangle_data(updated_data[0])
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    #refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    
    button.icon="check"
    button.button_style = 'success'
    button.description="Data updated"
    button.disabled=True
    return df_to_plot

apibutton2=wdg.Button(
    description='Update data',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to download current Public Health England data',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)
    

# remember to register your button callback function with the button
apibutton2.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton2)

# run all cells before clicking on this button

Button(description='Update data', icon='download', style=ButtonStyle(), tooltip='Click to download current Pub…

In [9]:
#updated_data=access_api()

This is the version of the data to display. 
Pay attention to the dates:
* current = stored data.
* updated = last available date from PHE.

In [19]:
try:
    df_to_plot=wrangle_data(updated_data[0])
    print('Your new COVID data is \033[1mupdated\033[0m up to: ' + '\033[1m'+ str(df_to_plot.index[-1]).split(' ')[0]+"\n")
    
except NameError as e:
    df_to_plot=df_to_plot
    print('The stored COVID data is\033[1m current\033[0m to: ' + '\033[1m'+ str(df_to_plot.index[-1]).split(' ')[0]+"\n")
    
display(df_to_plot)


The stored COVID data is current to: 2021-10-26



,East Midlands,East of England,London,North East,North West,South East,South West,West Midlands,Yorkshire and the Humber
2020-04-17,169,360,704,263,594,552,309,373,315
2020-04-18,138,344,689,158,550,414,226,327,370
2020-04-19,236,273,453,280,634,678,221,382,257
2020-04-20,97,256,297,216,457,315,192,360,150
2020-04-21,120,235,418,164,460,778,184,330,251
...,...,...,...,...,...,...,...,...,...
2021-10-22,3394,4796,3928,2037,5089,7347,6050,4295,3997
2021-10-23,3642,5353,4435,2353,4559,7365,6131,3698,3533
2021-10-24,2558,3618,3279,1610,3778,6250,4832,2838,3025
2021-10-25,2897,3891,3118,1430,3879,6258,4863,3465,2904


In [ ]:
#df_to_plot.columns

### 5. Visualise the data

Select the English region of interest.
* For more than one region, hold **Ctrl** button while selecting.

In [ ]:
def make_int_graph(data_to_plot):
    series=wdg.SelectMultiple(
        options=data_to_plot.columns,
        value=['East Midlands'],
        rows=9,
        description='Region:',
        disabled=False
    )

    scale=wdg.RadioButtons(
        options=['linear', 'log'],
    #    value='pineapple', # Defaults to 'pineapple'
    #    layout={'width': 'max-content'}, # If the items' names are long
        description='Scale:',
        disabled=False
    )

    # try replacing HBox with a VBox
    controls=wdg.HBox([series, scale]) #HBox horizontal display of buttons (V=vertical)

    def regions_graph(gcols, gscale):
        if gscale=='linear':
            logscale=False
        else:
            logscale=True
        ncols=len(gcols)
        if ncols>0:
            data_to_plot[list(gcols)].plot(logy=logscale)
            plt.show() # important - graphs won't update if this is missing 
        else:
            print("Click to select data for graph")
            print("(CTRL-Click to select more than one category)")

    # keep calling timeseries_graph(gcols=value_of_series, gscale=value_of_scale); 
    # capture output in widget graph   
    graph=wdg.interactive_output(regions_graph, {'gcols': series, 'gscale': scale})
    return display(controls, graph)

    

In [ ]:
make_int_graph(df_to_plot)

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*